# 2.4 Modelado Dimensional Básico\n\n- Objetivo: comprender dimensiones y hechos, granularidad y cardinalidad.\n- Prerrequisitos: tablas dim_* y fact_* creadas.\n- Ejercicios: validar unicidad, densidad actividad, gaps inventario.\n- Reto: mini-dimensión fecha y días sin ventas.\n- Errores comunes: mezclar granularidades, claves poco claras, sobre-normalizar dimensiones pequeñas.

Elementos del dataset:
- Dimensiones: clientes, productos, regiones.
- Hechos: ventas, inventario, suscripciones.
Claves sustitutas vs naturales, granularidad de la tabla de hechos (venta por línea).

In [ ]:
-- Granularidad: validar que cada venta_id es único
SELECT venta_id, COUNT(*) AS repeticiones
FROM dbo.fact_ventas
GROUP BY venta_id
HAVING COUNT(*) > 1;

In [ ]:
-- Cardinalidad: clientes vs ventas
SELECT COUNT(*) AS clientes, (SELECT COUNT(*) FROM dbo.fact_ventas) AS filas_ventas
FROM dbo.dim_clientes;

---

## 🧪 Ejercicios Guiados
- 🟢 Calcular razón (filas ventas / clientes) para densidad de actividad promedio.
- 🟠 Detectar productos sin registros en inventario (dim vs hecho con `LEFT JOIN`).
- 🔴 Derivar una 'mini dimensión de fecha' on-the-fly con CTE y detectar días sin ventas.

In [ ]:
SELECT CAST(v.filas AS DECIMAL(10,2))/c.clientes AS ventas_por_cliente
FROM (SELECT COUNT(*) AS filas FROM dbo.fact_ventas) v
CROSS JOIN (SELECT COUNT(*) AS clientes FROM dbo.dim_clientes) c;

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
LEFT JOIN dbo.fact_inventario fi ON p.producto_id = fi.producto_id
WHERE fi.producto_id IS NULL;

In [ ]:
-- Ejemplo idea (ajustar según motor):
-- WITH fechas AS (
--   SELECT MIN(fecha) AS fmin, MAX(fecha) AS fmax FROM dbo.fact_ventas
-- ), serie AS (
--   SELECT fmin AS fecha FROM fechas
--   UNION ALL
--   SELECT DATEADD(DAY,1,fecha) FROM serie s JOIN fechas f ON s.fecha < f.fmax
-- ) SELECT fecha FROM serie EXCEPT SELECT DISTINCT fecha FROM dbo.fact_ventas;

---

## ⚠️ Errores Comunes
- Mezclar granularidades (línea vs. encabezado) genera métricas incorrectas.
- No definir claves primarias claras dificulta JOINs y duplica datos.
- Sobre-normalizar dimensiones pequeñas complica consultas sin beneficio real.
- Ignorar gaps (productos sin inventario, días sin ventas) oculta problemas operativos.

---

## ✅ Conclusiones
- Granularidad define el nivel de detalle; cardinalidad mide relaciones entre tablas.
- Validar unicidad y detectar huérfanos/gaps asegura calidad del modelo.
- Dimensiones pequeñas pueden desnormalizarse para simplicidad sin pérdida funcional.

---

## 🚀 Aplicación Práctica
- Auditorías de modelo antes de ETL para detectar inconsistencias.
- Dashboards de gaps (productos sin movimiento, días sin actividad) para alertas operativas.
- Cálculos de densidad para priorizar clientes/productos activos.

---

## 💼 Perspectiva de Negocio
- Modelos bien diseñados aceleran consultas y reducen costos de infraestructura.
- Detectar gaps proactivamente previene stockouts y pérdida de ventas.
- Granularidad correcta permite drill-down sin duplicar datos ni agregar manualmente.

---

## 🔖 Pie Editorial

**Curso**: Fundamentos de SQL Server - Nivel 2  
**Módulo**: 2.4 Modelado Dimensional Básico  
**Versión**: 2.0 (Actualizado Enero 2025)  
**Autor**: lraigosov / LuisRai  
**Licencia**: Uso educativo - Atribución requerida

> 💡 Nota sobre IA: Este material fue estructurado con asistencia de modelos de lenguaje (OpenAI GPT-4, Anthropic Claude); el contenido fue validado y curado por especialistas para evitar alucinaciones y asegurar aplicabilidad práctica.

---
## Navegación
[⬅️ Anterior](03_subconsultas_avanzadas.ipynb) | [Siguiente ➡️](05_kpis_avanzados.ipynb)
---
